# 단어중요도 구하기
- 2020-2 Artifitial Intelligence  
- week1 Assignment  
- Updated 0911

In [1]:
# ready
import pandas as pd
import numpy as np

import os
import sys
import joblib
import warnings
warnings.filterwarnings(action='ignore')

from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

### Load data

* step1. 텍스트를 불러들일 때, 모두 소문자로 바꿔서 리스트로 저장한다.  
대소문자가 달라도 같은 단어로 인식하기 위함.

In [2]:
doc1 = []
with open("data/단어중요도구하기/문서1.txt","r") as f:
  for i in f.readlines():
    i = i.lower()     # 소문자 변환
    doc1.append(i[:-1])
doc1 = ' '.join(doc1); doc1

'konkuk university is one of the comprehensive private universities located in seoul and chungju. the seoul campus is located in the southeastern part of seoul, near the han river, and is served by a metro station of the same name. the seoul campus has 13 undergraduate colleges and 13 graduate schools, whereas the glocal campus in chungju is composed of 7 undergraduate colleges and 2 graduate schools. currently, 23,000 students are attending the 238,980m² seoul campus and 15,500 students attending the 31,763m² global campus in chungju, which is about an hour away from the seoul campus.'

In [3]:
# 함수로 만들어두기
def doc(data_path):
    doc = []
    with open(data_path,"r") as f:
        for i in f.readlines():
            i = i.lower()
            doc.append(i[:-1])
        doc = ' '.join(doc)
    return doc

In [4]:
# 문서1, 문서2, 문서3 모두 불러오기
doc1 = doc('data/단어중요도구하기/문서1.txt')
doc2 = doc('data/단어중요도구하기/문서2.txt')
doc3 = doc('data/단어중요도구하기/문서3.txt')

In [5]:
# 확인
doc2

'computer engineering (cpe) is a branch of engineering that integrates several fields of computer science and electronic engineering required to develop computer hardware and software. computer engineers usually have training in electronic engineering (or electrical engineering), software design, and hardware-software integration instead of only software engineering or electronic engineering. computer engineers are involved in many hardware and software aspects of computing, from the design of individual microcontrollers, microprocessors, personal computers, and supercomputers, to circuit design. this field of engineering not only focuses on how computer systems themselves work but also how they integrate into the larger picture'

* step2. 특수문자를 없애준다.  
상대적으로 단어중요도구하기라는 목적에 부합하지 않은 것 같아서 제거

In [6]:
import re
import string

doc1 = doc1.translate(str.maketrans('','',string.punctuation))
doc2 = doc2.translate(str.maketrans('','',string.punctuation))
doc3 = doc3.translate(str.maketrans('','',string.punctuation))
doc1

'konkuk university is one of the comprehensive private universities located in seoul and chungju the seoul campus is located in the southeastern part of seoul near the han river and is served by a metro station of the same name the seoul campus has 13 undergraduate colleges and 13 graduate schools whereas the glocal campus in chungju is composed of 7 undergraduate colleges and 2 graduate schools currently 23000 students are attending the 238980m² seoul campus and 15500 students attending the 31763m² global campus in chungju which is about an hour away from the seoul campus'

* step3. 단어를 하나씩 띄어쓰기 단위로 쪼갠다.  
tokenizing

In [7]:
words_doc1 = [w for w in doc1.split()]
words_doc2 = [w for w in doc2.split()]
words_doc3 = [w for w in doc3.split()]

words_doc1[:10]    # 확인

['konkuk',
 'university',
 'is',
 'one',
 'of',
 'the',
 'comprehensive',
 'private',
 'universities',
 'located']

* step4. 각 단어가 문서에 몇 번 나왔는지 세어본다.  
dictionary 로 만들고 다시 데이터프레임으로 저장해둔다.

In [8]:
from collections import Counter
freq_doc1 = Counter(words_doc1)
freq_doc2 = Counter(words_doc2)
freq_doc3 = Counter(words_doc3)

freq_doc3.keys()    # 확인

dict_keys(['artificial', 'intelligence', 'ai', 'sometimes', 'called', 'machine', 'is', 'demonstrated', 'by', 'machines', 'unlike', 'the', 'natural', 'displayed', 'humans', 'and', 'animals', 'leading', 'textbooks', 'define', 'field', 'as', 'study', 'of', 'intelligent', 'agents', 'any', 'device', 'that', 'perceives', 'its', 'environment', 'takes', 'actions', 'maximize', 'chance', 'successfully', 'achieving', 'goals', 'colloquially', 'term', 'often', 'used', 'to', 'describe', 'or', 'computers', 'mimic', 'cognitive', 'functions', 'associate', 'with', 'human', 'mind', 'such', 'learning', 'problem', 'solving'])

In [9]:
df_doc1 = pd.DataFrame.from_dict(freq_doc1,orient='index',columns=['tf'])
df_doc2 = pd.DataFrame.from_dict(freq_doc2,orient='index',columns=['tf'])
df_doc3 = pd.DataFrame.from_dict(freq_doc3,orient='index',columns=['tf'])
df_doc3.head(5)    # 확인

,tf
artificial,2
intelligence,5
ai,2
sometimes,1
called,1


* step5. tf-idf  
전체 문서군의 단어를 비교하기 위해서 모든 단어를 세어보는 데이터 프레임을 만든다.  
각각 tf 와 idf 를 계산해서 weight를 구한다.  
![tfidf](https://class101.dev/images/thumbnails/tf-idf.png)

In [16]:
# outer join 된 데이터 프레임
df = pd.merge(df_doc1, df_doc2,left_index=True, right_index=True,how='outer')
df = pd.merge(df, df_doc3, left_index=True, right_index=True,how='outer')
df.columns=['doc1_tf','doc2_tf','doc3_tf']

In [17]:
df['df'] = df.iloc[:,:3].notnull().sum(axis = 1)     # 단어가 나온 문서의 수
df['idf'] = np.log(3/df.df)      # n = 3
df = df.fillna(0)
df

,doc1_tf,doc2_tf,doc3_tf,df,idf
13,2.0,0.0,0.0,1,1.098612
15500,1.0,0.0,0.0,1,1.098612
2,1.0,0.0,0.0,1,1.098612
23000,1.0,0.0,0.0,1,1.098612
238980m²,1.0,0.0,0.0,1,1.098612
31763m²,1.0,0.0,0.0,1,1.098612
7,1.0,0.0,0.0,1,1.098612
a,1.0,1.0,0.0,2,0.405465
about,1.0,0.0,0.0,1,1.098612
achieving,0.0,0.0,1.0,1,1.098612


In [18]:
# weight 구하기
doc1_tfidf = pd.merge(df_doc1,df['idf'],left_index=True,right_index=True,how='left')
doc1_tfidf['weight'] = doc1_tfidf['tf']*doc1_tfidf['idf']

doc2_tfidf = pd.merge(df_doc2,df['idf'],left_index=True,right_index=True,how='left')
doc2_tfidf['weight'] = doc2_tfidf['tf']*doc2_tfidf['idf']

doc3_tfidf = pd.merge(df_doc3,df['idf'],left_index=True,right_index=True,how='left')
doc3_tfidf['weight'] = doc3_tfidf['tf']*doc3_tfidf['idf']

# 각 문서별 weight 상위 5개
print(doc1_tfidf.sort_values(by=['weight'], axis=0, ascending=False).head(5))
print(doc2_tfidf.sort_values(by=['weight'], axis=0, ascending=False).head(5))
print(doc3_tfidf.sort_values(by=['weight'], axis=0, ascending=False).head(5))

          tf       idf    weight
campus     6  1.098612  6.591674
seoul      6  1.098612  6.591674
chungju    3  1.098612  3.295837
colleges   2  1.098612  2.197225
graduate   2  1.098612  2.197225
             tf       idf    weight
engineering   8  1.098612  8.788898
computer      6  1.098612  6.591674
software      4  1.098612  4.394449
electronic    3  1.098612  3.295837
design        3  1.098612  3.295837
              tf       idf    weight
intelligence   5  1.098612  5.493061
its            3  1.098612  3.295837
machines       2  1.098612  2.197225
as             2  1.098612  2.197225
humans         2  1.098612  2.197225


In [21]:
# 각 문서별 weight 상위인 값 추출 (1이상)
print(doc1_tfidf.sort_values(by=['weight'], axis=0, ascending=False).query('weight > 1').index)
print(doc2_tfidf.sort_values(by=['weight'], axis=0, ascending=False).query('weight > 1').index)
print(doc3_tfidf.sort_values(by=['weight'], axis=0, ascending=False).query('weight > 1').index)

Index(['campus', 'seoul', 'chungju', 'colleges', 'graduate', 'schools',
       'students', 'attending', 'undergraduate', '13', 'located', 'in', '2',
       'away', 'hour', 'an', 'whereas', 'glocal', 'composed', '7', 'currently',
       '23000', 'about', 'which', '238980m²', 'global', '31763m²', '15500',
       'konkuk', 'name', 'has', 'universities', 'same', 'station', 'metro',
       'one', 'comprehensive', 'served', 'river', 'han', 'near', 'university',
       'part', 'southeastern', 'private'],
      dtype='object')
Index(['engineering', 'computer', 'software', 'electronic', 'design', 'how',
       'engineers', 'hardware', 'only', 'several', 'many', 'fields',
       'individual', 'microcontrollers', 'microprocessors', 'personal',
       'supercomputers', 'circuit', 'this', 'not', 'focuses', 'on', 'cpe',
       'systems', 'themselves', 'work', 'but', 'also', 'they', 'integrate',
       'into', 'larger', 'aspects', 'computing', 'involved', 'electrical',
       'science', 'integrates',

## 예시
weight 상위 단어들 중에 불용어 처리 대상이 많은 것 같아서 자세하게 살펴본다. 

In [24]:
# weight 값 예시
doc3_tfidf.sort_values(by=['weight'], axis=0, ascending=False)

,tf,idf,weight
intelligence,5,1.098612,5.493061
its,3,1.098612,3.295837
machines,2,1.098612,2.197225
as,2,1.098612,2.197225
humans,2,1.098612,2.197225
artificial,2,1.098612,2.197225
ai,2,1.098612,2.197225
that,4,0.405465,1.621860
mind,1,1.098612,1.098612
colloquially,1,1.098612,1.098612


In [25]:
doc3

'artificial intelligence ai sometimes called machine intelligence is intelligence demonstrated by machines unlike the natural intelligence displayed by humans and animals leading ai textbooks define the field as the study of intelligent agents any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals colloquially the term artificial intelligence is often used to describe machines or computers that mimic cognitive functions that humans associate with the human mind such as learning and problem solving'

심볼에 대한 전처리는 했지만 불용어 처리 없이 했기 때문에 weight 상위 단어들 중에 핵심 내용이 아닌 단어들이 많았다.